# Analyzing CIA Factbook Data Using SQL
Prepared by Amanda Morphew-Ulm  
Last Edited: 2020-08-08  
*An important note to the reader:
This report was created as a guided project during the [Dataquest.io](https://app.dataquest.io/) course **SQL Fundamentals**.*

## Introduction

In this project, we'll work with data from the [CIA World Factbook](https://www.cia.gov/library/publications/the-world-factbook/), a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like:

* population - The population as of 2015.
* population_growth - The annual population growth rate, as a percentage.
* area - The total land and water area.

Let's connect to our database file:

In [1]:
%%capture
%load_ext sql
%sql sqlite:///datasets/factbook.db

## Exploring the Data

In [2]:
%%sql
SELECT * FROM facts LIMIT 5;

 * sqlite:///datasets/factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


The above query shows us a the first five rows in the facts table, as well as the column headings. Our guided project description gives us the following information on those columns:

* name - The name of the country.
* area- The country's total area (both land and water).
* area_land - The country's land area in square kilometers.
* area_water - The country's waterarea in square kilometers.
* population - The country's population.
* population_growth- The country's population growth as a percentage.
* birth_rate - The country's birth rate, or the number of births a year per 1,000 people.
* death_rate - The country's death rate, or the number of death a year per 1,000 people.

## Identifying Outliers

Let's start by calculating some summary statistics and look for any outlier countries.

In [3]:
%%sql
SELECT 
    MIN(population) as 'Minimum Population', 
    MAX(population) as 'Maximum Population', 
    MIN(population_growth) as 'Minimum Population Growth', 
    MAX(population_growth) as 'Maximum Population Growth'
    FROM facts;

 * sqlite:///datasets/factbook.db
Done.


Minimum Population,Maximum Population,Minimum Population Growth,Maximum Population Growth
0,7256490011,0.0,4.02


A few things stick out from the summary statistics in the last query:

* There's a country with a population of 0
* There's a country with a population of 7256490011 (or more than 7.2 billion people)

Let's use subqueries to zoom in on just these countries without using the specific values.

In [4]:
%%sql
SELECT * FROM facts 
 WHERE population = (SELECT MIN(population) FROM facts) 
 OR population = (SELECT MAX(population) FROM facts);

 * sqlite:///datasets/factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


It seems like the table contains a row for the whole world, which explains the population of over 7.2 billion. It also seems like the table contains a row for Antarctica, which explains the population of 0. This seems to match the [CIA Factbook page for Antarctica](https://www.cia.gov/library/publications/the-world-factbook/geos/ay.html).

Let's recompute the summary statistics we found earlier, and exclude the row for the whole world.

In [5]:
%%sql
SELECT
    MIN(population) as 'Minimum Population', 
    MAX(population) as 'Maximum Population', 
    MIN(population_growth) as 'Minimum Population Growth', 
    MAX(population_growth) as 'Maximum Population Growth' 
    FROM facts
    WHERE name <> 'World';

 * sqlite:///datasets/factbook.db
Done.


Minimum Population,Maximum Population,Minimum Population Growth,Maximum Population Growth
0,1367485388,0.0,4.02


## Exploring Population Density

Let's also explore the average values for the population and area columns - again, excluding the line for the whole world:

In [6]:
%%sql
SELECT 
    AVG(population) as 'Average Population', 
    AVG(area) as 'Average Area'
    FROM facts
    WHERE name <> 'World';

 * sqlite:///datasets/factbook.db
Done.


Average Population,Average Area
32242666.56846473,555093.546184739


Finally, let's build on the query we wrote for the previous screen to find countries that are densely populated. We'll identify countries that have:

* Above average values for population.
* Below average values for area.

It should not be necessary to manually exclude the row for the whole world, as it will not have a below average value for area.

In [7]:
%%sql
SELECT *
    FROM facts
    WHERE population > (
        SELECT AVG(population) 
        FROM facts
        )
    AND area < (
        SELECT AVG(area)
        FROM facts
        );

 * sqlite:///datasets/factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
173,th,Thailand,513120,510890,2230,67976405,0.34,11.19,7.8,0.0
185,uk,United Kingdom,243610,241930,1680,64088222,0.54,12.17,9.35,2.54
192,vm,Vietnam,331210,310070,21140,94348835,0.97,15.96,5.93,0.3


## Further Exploration

This marks the end of our guided project; however, there are some additional data questions presented here for optional further analysis.

### What countries have the highest population/area ratio and how does it compare to the list we found in the previous query?

In [8]:
%%sql
SELECT name, area, population, CAST(population AS FLOAT)/CAST(area AS FLOAT) as pop_area_ratio
    FROM facts
    ORDER BY pop_area_ratio DESC
    LIMIT 10;

 * sqlite:///datasets/factbook.db
Done.


name,area,population,pop_area_ratio
Macau,28,592731,21168.964285714286
Monaco,2,30535,15267.5
Singapore,697,5674472,8141.279770444763
Hong Kong,1108,7141106,6445.041516245487
Gaza Strip,360,1869055,5191.819444444444
Gibraltar,6,29258,4876.333333333333
Bahrain,760,1346613,1771.8592105263158
Maldives,298,393253,1319.6409395973155
Malta,316,413965,1310.01582278481
Bermuda,54,70196,1299.9259259259259


This list is interesting because there is zero overlap with our previous list of countries with above-average population and below-average area values. We can see that this list of countries all have populations under the average value of about 32 million. However, because their area values are all drastically smaller than the average area of about half a million square kilometers, that pushes each of their population-to-area ratios into the top ten values for this table.

### What country has the most people? What country has the highest growth rate?

In [9]:
%%sql
SELECT name as 'The Country With the Most People Is', population as 'With a Population Of'
    FROM facts
    WHERE name <> 'World'
    ORDER BY population DESC
    LIMIT 1;

 * sqlite:///datasets/factbook.db
Done.


The Country With the Most People Is,With a Population Of
China,1367485388


In [10]:
%%sql
SELECT name as 'The Country With the Highest Growth Rate Is', population_growth as 'With a Population Growth Of'
    FROM facts
    WHERE name <> 'World'
    ORDER BY population_growth DESC
    LIMIT 1;

 * sqlite:///datasets/factbook.db
Done.


The Country With the Highest Growth Rate Is,With a Population Growth Of
South Sudan,4.02


### Which countries have the highest ratios of water to land? Which countries have more water than land?

In [11]:
%%sql
SELECT name, area_land, area_water, CAST(area_water AS FLOAT)/CAST(area_land AS FLOAT) as ratio_water_to_land
    FROM facts
    WHERE name <> 'World'
    ORDER BY ratio_water_to_land DESC
    LIMIT 10;

 * sqlite:///datasets/factbook.db
Done.


name,area_land,area_water,ratio_water_to_land
British Indian Ocean Territory,60,54340,905.6666666666666
Virgin Islands,346,1564,4.520231213872832
Puerto Rico,8870,4921,0.5547914317925592
"Bahamas, The",10010,3870,0.3866133866133866
Guinea-Bissau,28120,8005,0.2846728307254623
Malawi,94080,24404,0.25939625850340137
Netherlands,33893,7650,0.22571032366565366
Uganda,197100,43938,0.22292237442922375
Eritrea,101000,16600,0.16435643564356436
Liberia,96320,15049,0.15623961794019933


Here we can see that our top two water-to-land ratios are greater than 1, meaning that these two countries have more water than land. This makes sense for a territory named British Indian Ocean Territory, and for a territory of islands such as the Virgin Islands. Other countries in this list, such as the Netherlands, Eritrea, and Liberia, have extensive coastlines, while a considerable portion of Uganda's total area is made up of Lakes Victoria, Albert, Edward, and Kyoga.

### Which countries have a higher death rate than birth rate?

In [12]:
%%sql
SELECT *
    FROM facts
    WHERE death_rate > birth_rate
    AND name <> 'World'
    ORDER BY name;

 * sqlite:///datasets/factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
10,au,Austria,83871,82445,1426,8665550,0.55,9.41,9.42,5.56
16,bo,Belarus,207600,202900,4700,9589689,0.2,10.7,13.36,0.7
22,bk,Bosnia and Herzegovina,51197,51187,10,3867055,0.13,8.87,9.75,0.38
26,bu,Bulgaria,110879,108489,2390,7186893,0.58,8.92,14.44,0.29
44,hr,Croatia,56594,55974,620,4464844,0.13,9.45,12.18,1.39
47,ez,Czech Republic,78867,77247,1620,10644842,0.16,9.63,10.34,2.33
57,en,Estonia,45228,42388,2840,1265420,0.55,10.51,12.4,3.6
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
67,gr,Greece,131957,130647,1310,10775643,0.01,8.66,11.09,2.32
75,hu,Hungary,93028,89608,3420,9897541,0.22,9.16,12.73,1.33


### Which countries will add the most people to their population next year?

At first glance, it may seem as though the highest population_growth values should give us those country names:

In [13]:
%%sql
SELECT *
    FROM facts
    WHERE name <> 'World'
    ORDER BY population_growth DESC
    LIMIT 10;

 * sqlite:///datasets/factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
162,od,South Sudan,644329,None,None,12042910,4.02,36.91,8.18,11.47
106,mi,Malawi,118484,94080,24404,17964697,3.32,41.56,8.41,0.0
29,by,Burundi,27830,25680,2150,10742276,3.28,42.01,9.27,0.0
128,ng,Niger,None,1266700,300,18045729,3.25,45.45,12.42,0.56
182,ug,Uganda,241038,197100,43938,37101745,3.24,43.79,10.69,0.74
141,qa,Qatar,11586,11586,0,2194817,3.07,9.84,1.53,22.39
27,uv,Burkina Faso,274200,273800,400,18931686,3.03,42.03,11.72,0.0
109,ml,Mali,1240192,1220190,20002,16955536,2.98,44.99,12.89,2.26
219,cw,Cook Islands,236,236,0,9838,2.95,14.33,8.03,None
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62


However, if we calculate estimated births for next year, keeping in mind that the birth rate is the number of births a year per 1000 people, we see a completely different list:

In [14]:
%%sql
SELECT *, (CAST(population AS FLOAT) / 1000) * CAST(birth_rate AS FLOAT) AS numerical_birth_rate
    FROM facts
    WHERE name <> 'World'
    ORDER BY numerical_birth_rate DESC
    LIMIT 10;
    

 * sqlite:///datasets/factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,numerical_birth_rate
77,in,India,3287263,2973193,314070,1251695584,1.22,19.55,7.32,0.04,24470648.667200003
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44,17079892.496120002
129,ni,Nigeria,923768,910768,13000,181562056,2.45,37.64,12.9,0.22,6833995.78784
197,ee,European Union,4324782,None,None,513949445,0.25,10.2,10.2,2.5,5242284.339
132,pk,Pakistan,796095,770875,25220,199085847,1.46,22.58,6.49,1.54,4495358.42526
78,id,Indonesia,1904569,1811569,93000,255993674,0.92,16.72,6.37,1.16,4280214.22928
186,us,United States,9826675,9161966,664709,321368864,0.78,12.49,8.15,3.86,4013897.11136
58,et,Ethiopia,1104300,None,104300,99465819,2.89,37.27,8.19,0.22,3707091.0741300005
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46,3571766.7293
24,br,Brazil,8515770,8358140,157630,204259812,0.77,14.46,6.58,0.14,2953596.8815200003


This demonstrates the importance of understanding that:
* because it is per 1000 people, the birth rate results in a much larger number of people being born in a country, for countries with very large populations
* for a country with a very small population, such as the Cook Islands, it doesn't take a large number of births to have a significant effect on population growth
* our numerical_birth_rate calculated value in the above table also does not account for deaths

Let's reformulate this query to also calculate the projected number of deaths based on the death rate and population, as well as the difference between the number of project births and deaths, so we can see the expected change in population:

In [15]:
%%sql
SELECT name, population, population_growth, birth_rate, death_rate, 
    (CAST(population AS FLOAT) / 1000) * CAST(birth_rate AS FLOAT) AS numerical_birth_rate, 
    (CAST(population AS FLOAT) / 1000) * CAST(death_rate AS FLOAT) AS numerical_death_rate,
    ((CAST(population AS FLOAT) / 1000) * CAST(birth_rate AS FLOAT)) - 
        ((CAST(population AS FLOAT) / 1000) * CAST(death_rate AS FLOAT)) AS numerical_pop_change
    FROM facts
    WHERE name <> 'World'
    ORDER BY numerical_pop_change DESC
    LIMIT 10;

 * sqlite:///datasets/factbook.db
Done.


name,population,population_growth,birth_rate,death_rate,numerical_birth_rate,numerical_death_rate,numerical_pop_change
India,1251695584,1.22,19.55,7.32,24470648.667200003,9162411.67488,15308236.992320003
China,1367485388,0.45,12.49,7.53,17079892.496120002,10297164.97164,6782727.524480002
Nigeria,181562056,2.45,37.64,12.9,6833995.78784,2342150.5224,4491845.26544
Pakistan,199085847,1.46,22.58,6.49,4495358.42526,1292067.14703,3203291.27823
Ethiopia,99465819,2.89,37.27,8.19,3707091.0741300005,814625.05761,2892466.0165200005
Indonesia,255993674,0.92,16.72,6.37,4280214.22928,1630679.70338,2649534.5259
Bangladesh,168957745,1.6,21.14,5.61,3571766.7293,947852.9494500001,2623913.7798499996
"Congo, Democratic Republic of the",79375136,2.45,34.88,10.07,2768604.7436800003,799307.6195200001,1969297.1241600001
Philippines,100998376,1.61,24.27,6.11,2451230.58552,617100.0773600001,1834130.5081600002
Mexico,121736809,1.18,18.78,5.26,2286217.27302,640335.6153399999,1645881.6576800002


Here we see the United States, the European Union, and Brazil fall out of the top ten. This makes sense for the European Union, as the death rate value is equal to the birth rate value. For the United States and Brazil, we see a combination of factors contributing to this change: the death rate to birth rate ratio is either relatively higher compared to countries with similar populations within that numerical_birth_rate top ten (for example, the US as compared to Indonesia), or their populations are considerably lower than other countries with a similar death rate to birth rate ratio (for example, the US as compared to China).

## Conclusion

SQL queries are extremely useful for exploring, analyzing, organizing, and summarizing data. However, it's important to understand the data set columns that you have to work with, and how those values can interact with one another and affect your query results.